In [ ]:
from acdh_tei_pyutils.tei import TeiReader
from tqdm import tqdm
from icecream import ic
from normdata.utils import import_from_normdata

In [ ]:
source_file = "https://github.com/hermann-bahr/bahr-textverzeichnis-data/raw/main/data/indices/listperson.xml"

In [ ]:
doc = TeiReader(source_file)

In [ ]:
nsmap = doc.nsmap
bahr_url = "https://hermanbahrtextverzeichnis/"
bahr_domain = "hermanbahrtextverzeichnis"
bahr_col, _ = Collection.objects.get_or_create(name="Bahr Textverzeichnis")

In [ ]:
no_gnd = set()
no_wikidata = set()
not_created = set()
for x in tqdm(doc.any_xpath(".//tei:person")):
    entity = False
    hbtv_uri = x.xpath(".//tei:idno[@type='HB-tv']/text()", namespaces=nsmap)[0]
    hbtv_url = f"{bahr_url}{hbtv_uri}"
    try:
        gnd = x.xpath(".//tei:idno[@type='gnd']/text()", namespaces=nsmap)[0]
        entity = import_from_normdata(gnd, 'person')
    except IndexError:
        no_gnd.add(hbtv_uri)
        try:
            wikidata = x.xpath(".//tei:idno[@type='wikidata']/text()", namespaces=nsmap)[0]
            entity = import_from_normdata(gnd, 'person')
        except IndexError:
            no_wikidata.add(hbtv_uri)
            not_created.add(hbtv_uri)
    if entity:
        entity.collection.add(bahr_col)
        uri, _ = Uri.objects.get_or_create(uri=hbtv_url, domain=bahr_domain)
        uri.entity = entity
        uri.save()
   

In [ ]:
len(not_created)